In [16]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.models import load_model
from keras.initializers import glorot_normal, Zeros, Ones
import keras.backend as K
from keras.optimizers import RMSprop
import tensorflow as tf

In [55]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-dwws_xab
  Stored in directory: /tmp/pip-ephem-wheel-cache-guzfwt5w/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [56]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

from keras_contrib.layers import CRF
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [57]:
%%time
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import random

from sklearn import preprocessing
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.decomposition import PCA
from math import sqrt
from scipy import stats
from scipy.stats import norm, skew #for some statistics

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.39 ms


In [58]:
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [59]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [60]:
import os
os.listdir()

['__notebook_source__.ipynb', '.ipynb_checkpoints']

In [61]:
train = pd.read_csv('../input/train_3pirksi/train.csv')
test = pd.read_csv('../input/test_xev14ad/test.csv')
subm = pd.read_csv('../input/sample_submission_usrypcc/sample_submission.csv')

In [62]:
test.head()

,id,Doc_ID,Sent_ID,Word
0,4543834,30001,191283,CCCVA
1,4543835,30001,191283,","
2,4543836,30001,191283,MANOVA
3,4543837,30001,191283,","
4,4543838,30001,191283,my


In [63]:
train['tag'].value_counts()

O                4446206
B-indications      53003
I-indications      44624
Name: tag, dtype: int64

In [64]:
train[train['tag'].isin(['B-indications','I-indications'])]

,id,Doc_ID,Sent_ID,Word,tag
171,172,1,8,strategies,B-indications
211,212,2,10,MICROCEPHALIA,B-indications
212,213,2,10,VERA,I-indications
233,234,3,12,reactive,B-indications
234,235,3,12,hyperemia,I-indications
1328,1329,9,63,acute,B-indications
1329,1330,9,63,cholecystitis,I-indications
1347,1348,10,65,Temporal,B-indications
1348,1349,10,65,lobe,I-indications
1349,1350,10,65,epilepsy,I-indications


In [65]:
train.shape
test.shape

(4543833, 5)

(2994463, 4)

In [66]:
train.nunique()

id         4543833
Doc_ID       30000
Sent_ID     191282
Word        184505
tag              3
dtype: int64

In [67]:
words = list(set(train["Word"].values))
words.append("ENDPAD")

In [68]:
n_words = len(words); n_words

184507

In [69]:
tags = list(set(train["tag"].values))

In [70]:
n_tags = len(tags); n_tags

3

In [71]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [72]:
# agg_func = lambda s: [(w, t) for w, t in zip(train["Word"].values.tolist(),train["tag"].values.tolist())]
# grouped = train[:500].groupby("Sent_ID").apply(agg_func)
# grouped[1]
# sentences = [s for s in grouped]

In [73]:
getter = SentenceGetter(train)

In [74]:
sent = getter.get_next()

In [75]:
print(sent)

[('Obesity', 'O'), ('in', 'O'), ('Low-', 'O'), ('and', 'O'), ('Middle-Income', 'O'), ('Countries', 'O'), (':', 'O'), ('Burden', 'O'), (',', 'O'), ('Drivers', 'O'), (',', 'O'), ('and', 'O'), ('Emerging', 'O'), ('Challenges', 'O'), ('.', 'O')]


In [76]:
sentences = getter.sentences

In [77]:
len(sentences)

191282

In [78]:
max_len = 50
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [79]:
word2idx["We"]

178230

In [80]:
tag2idx["O"]

1

In [81]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [82]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)

In [83]:
X[1]

array([178230,  39561, 117230, 101795, 148971,  24343, 107959,  80424,
       137615, 127788, 182216, 146110, 127788,  36765, 111847,  33813,
        36765, 180420,  40880, 127788,   1016, 145034,   1016, 102213,
       174200,  36765, 141190,  96076, 137911, 135435,  42658,  66414,
        36765, 113746,  69122, 172849,  97321, 136412,  52680, 184506,
       184506, 184506, 184506, 184506, 184506, 184506, 184506, 184506,
       184506, 184506], dtype=int32)

In [84]:
y = [[tag2idx[w[1]] for w in s] for s in sentences]

In [85]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [86]:
y[1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1], dtype=int32)

In [87]:
from keras.utils import to_categorical

In [88]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [89]:
from sklearn.model_selection import train_test_split

In [90]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [91]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [92]:
model = Model(input, out)

In [93]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

In [94]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=2,
                    validation_split=0.1, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 154937 samples, validate on 17216 samples
Epoch 1/1
154937/154937 [==============================] - 635s 4ms/step - loss: 0.0203 - crf_viterbi_accuracy: 0.9928 - val_loss: 0.0128 - val_crf_viterbi_accuracy: 0.9939


In [97]:
!pip install seqeval

In [98]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [96]:
test_pred = model.predict(X_te, verbose=1)

19129/19129 [==============================] - 14s 723us/step


In [99]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)

In [ ]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

In [ ]:
print(classification_report(test_labels, pred_labels))

In [100]:
i = 1927
p = model.predict(np.array([X_te[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_te[i], -1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_te[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-1], tags[t], tags[pred]))

Word           ||True ||Pred
SAEs           : O     O
antiproliferation: O     O
α2A-adrenoceptors: O     O
microestrutura : O     O
genuinely      : O     O
pericyte       : O     O
lanigera       : O     O
neuro-biological: O     O
cross-correlated: O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           : O     O
País           

In [101]:
test_sentence = ["type", "2", "diabetes", "Fellow", "of", "the", "Royal", "Society", ",", "a", "lifetime", "member",
                 "of", "the", "Pontifical", "Academy", "of", "Sciences", ",", "and", "a", "recipient", "of",
                 "the", "Presidential", "Medal", "of", "type", "2", "diabetes", "highest", "civilian", "award",
                 "in", "the", "United", "States", "."]

In [107]:
class SentenceGetterTest(object):
    
    def __init__(self, data):
        self.n_sent = 191283
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w) for w in s["Word"].values.tolist()]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [108]:
testgetter = SentenceGetterTest(test)
test_sentence = testgetter.sentences

In [110]:
test_sentence

[['CCCVA', ',', 'MANOVA', ',', 'my', 'black', 'hen', '.'],
 ['Comments', 'on', 'repeated', 'measures', '.'],
 ['Nikolsky',
  'sign',
  ';',
  'page',
  'from',
  'notable',
  'contributors',
  'to',
  'the',
  'knowledge',
  'of',
  'dermatology',
  '.'],
 ['[',
  'Obesity',
  'as',
  'a',
  'concomitant',
  'cause',
  'in',
  'the',
  'complex',
  'etiology',
  'of',
  'arteriosclerosis',
  ']',
  '.'],
 ['Tropical',
  'mixtures',
  'of',
  'star',
  'tree',
  'metrics',
  'We',
  'study',
  'tree',
  'metrics',
  'that',
  'can',
  'be',
  'realized',
  'as',
  'a',
  'mixture',
  'of',
  'two',
  'star',
  'tree',
  'metrics',
  '.'],
 ['We',
  'prove',
  'that',
  'the',
  'only',
  'trees',
  'admitting',
  'such',
  'a',
  'decomposition',
  'are',
  'the',
  'ones',
  'coming',
  'from',
  'a',
  'tree',
  'with',
  'at',
  'most',
  'one',
  'internal',
  'edge',
  ',',
  'and',
  'whose',
  'weight',
  'satisfies',
  'certain',
  'linear',
  'inequalities',
  '.'],
 ['We',
  '

In [111]:
x_test_sent  = pad_sequences(maxlen=max_len, sequences= [[word2idx.get(w, 0) for w in s] for s in test_sentence], padding="post", value=n_words - 1)

In [112]:
p = model.predict(np.array([x_test_sent[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test_sentence, p[0]):
    print("{:15}: {:5}".format(w, tags[pred]))

Word           ||Prediction


TypeError: unsupported format string passed to list.__format__

In [113]:
words = []
predss=[]
for i in range(len(x_test_sent)):
    if i%10000 ==0:
        print(i)
    idxs = i
    p = model.predict(np.array([x_test_sent[idxs]]))
    p = np.argmax(p, axis=-1)
    #print(test_sentence[idxs])
#     print(tags[p[0]])
    for w, pred in zip(test_sentence[idxs], p[0]):
        #print(w, tags[pred])
        words.append(w)
        predss.append(tags[pred])
    break

0


In [115]:
words
predss

['CCCVA', ',', 'MANOVA', ',', 'my', 'black', 'hen', '.']

['B-indications', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [ ]:
subm['tag']=predss
subm.to_csv('submission_nn.csv',index=False)